In [2]:
import mne 
import os
import mne 
from mne.io import read_raw
import numpy as np
import pandas as pd

In [5]:
data_folder = "data/"
freq = 128
q = 256/freq

# Set the path to the patient folder
data_files = sorted([f for f in os.listdir(data_folder) if f.startswith('chb')])
# Define the list of channels to keep
# channels_to_keep = ['FP1-F7', 'FP1-F3', 'FP2-F4', 'F8-T8','FP2-F8', 'T8-P8-1', 'T8-P8-0', 'P8-O2', 'P4-O2']# 
channels_to_keep = ['FP1-F7', 'FP1-F3']# 
dataframes = []  # List to store individual DataFrames

for folder_list in range(1,len(data_files[:5])):  # Assumes folders are named ch01 to ch10
        folder_name = f'chb{folder_list:02d}'  # Format the folder name as ch01, ch02, etc.
        folder_path = os.path.join(data_folder,folder_name)  # Create the full path to the folder

        # Create an empty list to store raw data
        patient_raws = []

        # Create an empty list to store file names
        file_names = []

        # Get a list of .edf files sorted by filename numbers
        edf_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.edf')])

        # Read the first .edf file to obtain the channel information
        first_file_path = os.path.join(folder_path, edf_files[0])
        first_raw = mne.io.read_raw_edf(first_file_path, preload=False, verbose="ERROR")
        first_raw.pick_channels(channels_to_keep)

        # Loop through the sorted .edf files
        for file in edf_files:
            file_path = os.path.join(folder_path, file)
            raw = mne.io.read_raw_edf(file_path, preload=False, verbose="ERROR")
            #Keep only the specified channels
            raw.pick_channels(channels_to_keep)
            raw_time_df = raw.to_data_frame()
            new_time = raw_time_df["time"].iloc[::2].reset_index()
            #raw_time_df = raw_time_df["time"]
            raw2 = raw.resample(sfreq=freq)

            # Set the channel information for each Raw object to match the first Raw object
            raw2.info = first_raw.info
            raw_df = raw2.to_data_frame()
            raw_df["file_name"] = file
            raw_df["original_time"] = new_time["time"]
            patient_raws.append(raw_df)
            #file_names.extend([file] * len(raw.times))

        # Concatenate the raw objects into a single patient_combined object while maintaining a continuous time axis
        #patient_combined = mne.concatenate_raws(patient_raws, preload=False)
        patient_combined = pd.concat(patient_raws, ignore_index=True)
        # Print info about the combined raw object for this patient
        #print(patient_combined.info)

        #df = patient_combined.to_data_frame()
        #df['file_name'] = file_names
        dataframes.append(patient_combined)
# Concatenate all DataFrames into a single DataFrame
final_dataframe = pd.concat(dataframes, ignore_index=True)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy functi

In [6]:
# Define the folder where your text files are located
source_path = "data/"

# Initialize an empty list to store data
all_data = []

# Loop through subfolders in the main data folder
for folder_name in os.listdir(source_path):
    folder_path = os.path.join(source_path, folder_name)

    # Check if the item in the main folder is a directory (subfolder)
    if os.path.isdir(folder_path):
        # Loop through the text files in the subfolder
        for filename in os.listdir(folder_path):
            if filename.endswith(".txt"):
                file_path = os.path.join(folder_path, filename)
                
                # Extract the part of the filename before the hyphen as the "Source File" value
                source_file = filename.split("-")[0]
                
                # Initialize a list to store data for each file
                data = []
                
                # # Initialize a dictionary to hold the current data
                current_data = {"File Name": None, "Number of Seizures": None, "Seizure Start Time": np.nan, "Seizure End Time": np.nan, "Source File": source_file}
                
                # Open and read the current text file
                with open(file_path, "r") as file:
                    for line in file:
                        line = line.strip()
                        if line.startswith("File Name:"):
                            current_data = {"File Name": None, "Number of Seizures": None, "Seizure Start Time": np.nan, "Seizure End Time": np.nan, "Source File": source_file}
                            current_data["File Name"] = line.split(": ", 1)[1]
                        elif line.startswith("Number of Seizures in File:"):
                            current_data["Number of Seizures"] = line.split(": ",1)[1]
                            if current_data["Number of Seizures"] == 0:
                                data.append(current_data.copy())
                        elif line.startswith("Seizure Start Time:"):
                            current_data["Seizure Start Time"] = int(line.split(": ", 1)[1].split()[0])
                        elif line.startswith("Seizure End Time:"):
                            current_data["Seizure End Time"] = int(line.split(": ", 1)[1].split()[0])
                            data.append(current_data.copy())
                
                all_data.extend(data)

# Create a DataFrame from the collected data
seizure_seq = pd.DataFrame(all_data)
seizure_seq = seizure_seq.rename(columns={"File Name": "file_name"})
seizure_seq["Seizure Start Time"] = seizure_seq["Seizure Start Time"].astype(float)
seizure_seq["Seizure End Time"] = seizure_seq["Seizure End Time"].astype(float)

# Display the DataFrame
print(seizure_seq)

        file_name Number of Seizures  Seizure Start Time  Seizure End Time  \
0    chb03_01.edf                  1               362.0             414.0   
1    chb03_02.edf                  1               731.0             796.0   
2    chb03_03.edf                  1               432.0             501.0   
3    chb03_04.edf                  1              2162.0            2214.0   
4    chb03_34.edf                  1              1982.0            2029.0   
5    chb03_35.edf                  1              2592.0            2656.0   
6    chb03_36.edf                  1              1725.0            1778.0   
7    chb04_05.edf                  1              7804.0            7853.0   
8    chb04_08.edf                  1              6446.0            6557.0   
9    chb05_06.edf                  1               417.0             532.0   
10   chb05_13.edf                  1              1086.0            1196.0   
11   chb05_16.edf                  1              2317.0        

In [7]:
def assign_seizure_flag(final_df, seizures_df):
    # Create a new column "Seizure" and initialize it to False
    final_df["Seizure"] = 0
    
    # Iterate through each row in participants_df and check if 'original_time' is within any seizure range
    for index, row in final_df.iterrows():
        file_name = row["file_name"]
        original_time = row["original_time"]
        matching_seizure = seizures_df[(seizures_df["file_name"] == file_name) & (seizures_df["Seizure Start Time"] <= original_time) & (seizures_df["Seizure End Time"] >= original_time)]
        
        if not matching_seizure.empty:
            final_df.at[index, "Seizure"] = 1
    
    return final_df

In [8]:
final_dataframe_new = assign_seizure_flag(final_dataframe, seizure_seq)

KeyboardInterrupt: 